NLP 처리과정 
토큰화 -> 정체,추출 -> 인코딩

# 1.표준 토큰화
자연어 처리에 사용되는 대표적인 파이썬 패키지는 NLTK

1.1 표준 토큰화
treebank 사용 


In [ ]:
!pip install nltk

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer  = TreebankWordTokenizer()
text = "Model-based RL don't need a value function for the policy"
print(tokenizer.tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


1.2 토큰화 라이브러리
여러종류의 tokenizer 가 있음 word_tokenizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(text))

['Model-based', 'RL', 'do', "n't", 'need', 'a', 'value', 'function', 'for', 'the', 'policy']


# 2. 어간 추출 및 표제어 추출
단어의 형태소 level에서 분석을 하게 되면 다른 품사 또는 다른 시제의 단어라고 해도 같은 형태로 토큰화 가능

-->둘의 차이 : 품사의 태깅 유무

-->표제어 : v, n 태깅 가능

-->어간 : 태깅 불가능

2.1 어간 추출Stemmer vs 표제어 추출lemmazation 

대표적 어간 추출 기법은 porter 추출 패키지

In [ ]:
#어간 추출
from nltk.stem import PorterStemmer, LancasterStemmer
stem1 = PorterStemmer()
stem2 = LancasterStemmer()
words = ["eat", "ate", "eaten", "eating"]
print("Poter Stemmer :", [stem1.stem(w) for w in words])
print("Lancaster Stemmer :", [stem2.stem(w) for w in words]) #왜 ate를 at으로..?

Poter Stemmer : ['eat', 'ate', 'eaten', 'eat']
Lancaster Stemmer : ['eat', 'at', 'eat', 'eat']


In [ ]:
#표제어 추출
from nltk import WordNetLemmatizer
nltk.download('wordnet')
lemm = WordNetLemmatizer()
words = ["eat", "ate", "eaten", "eating"]
print('WordNet Lemmetizer  :', [lemm.lemmatize(w, pos='v') for w in words]) #pos로 단어의 품사 지정해줌
#모든 시제를 동사원형으로 바꿔줌 품사태깅이 가능하다면 표제어 추출이 좋음

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
WordNet Lemmetizer  : ['eat', 'eat', 'eat', 'eat']


# 3. 불용어 제거
3.1불용어 예시

영어의 불용어 예시 

stopword단어 데이터를 받기 위한 사전작업이 필요함

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english')[:5])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we']


In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

input_sentence = "we should all study hard for the exam."
stop_words = set(stopwords.words('english'))

#토큰화 후 불용어제거 for문

word_tokens = word_tokenize(input_sentence)
result = []
for w in word_tokens:
  if w not in stop_words:
    result.append(w)

print(word_tokens)
print(result)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['we', 'should', 'all', 'study', 'hard', 'for', 'the', 'exam', '.']
['study', 'hard', 'exam', '.']


# 4. 정수 인코딩 및 sorting
4.1 Enumerate 사용 

In [ ]:
mylist = ['Enflish', 'math','Science' ]
for n, name in enumerate(mylist):
  print('Course : {}, Number : {}'.format(name,n))

  

Course : Enflish, Number : 0
Course : math, Number : 1
Course : Science, Number : 2


4.2 정수 인코딩 및 High-frequency Sorting

In [ ]:
vocab = {'apple':2,'July':6,'piano':4,'cup':8,'orange':1} #BoW
vocab_sort = sorted(vocab.items(), key= lambda x:x[1], reverse = True) #슛자 기준 역순 정렬
print(vocab_sort)

# 많이 사용된 벡터에 1을 부여하고 싶음 -> word[0] = cup, index enumerate하면서 1씩 증가
word2inx = {word[0] : index +1 for index, word in enumerate(vocab_sort)}
print(word2inx)

[('cup', 8), ('July', 6), ('piano', 4), ('apple', 2), ('orange', 1)]
{'cup': 1, 'July': 2, 'piano': 3, 'apple': 4, 'orange': 5}


Bow로 만들어진 토큰화의 결과(Bow)를 가장 높은 빈도수부터 재정렬하고 이를 통해 정수 인코딩 진행

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
text = "Model-based RL don't need a value function for the policy."\
       "but some of Model-based RL algorithms do have a value function."
token_text = tokenizer.tokenize(text)
word2inx = {}
Bow = []
for word in token_text:
  if word not in word2inx.keys():
    word2inx[word] = len(word2inx) #비어있는 상태의 딕셔너리 길이 즉 0부터 시작 ex> Model = 0 
    Bow.insert(len(word2inx)-1,1) #(0,1)에 insert
  else:
    inx= word2inx.get(word) #get함수의 의미 : 기존에 있는 word를 가져오라는 의미
    Bow[inx] +=1 #가져온 word값에 +1

print(word2inx) # inx
print(Bow) #빈도수를 고려해서 인코딩


{'Model-based': 0, 'RL': 1, 'do': 2, "n't": 3, 'need': 4, 'a': 5, 'value': 6, 'function': 7, 'for': 8, 'the': 9, 'policy.but': 10, 'some': 11, 'of': 12, 'algorithms': 13, 'have': 14, '.': 15}
[2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]


문장으로부터  토큰화를 통해 토큰 리스트를 만들고, 이를 이용해서 BoW를 생성하는 전체 알고리즘 word2inx = {}를 만들고, 리스트에 없는 단어의 경우 새로 리스트와 BoW에 단어를 추가하고 리스트에 있는 단어는 inx +=1

# 5. 유사도 분석

5.1코사인 유사도

In [1]:
import numpy as np
# 두 벡터 사이의 각도의 코사인값이 코사인 유사도 cf>cos0=1, cos90 = 0, cos180 = -1
# 내적 = a,b의 norm값 x cos0      *이때 내적은 각 성분끼리의 곱의 합산
def cos_sim(A,B):
  return np.dot(A,B) / (np.linalg.norm(A)*np.linalg.norm(B))

a = [1,0,0,1]
b = [0,1,1,0]
c = [1,1,1,1]
print(cos_sim(a,b), cos_sim(b,c), cos_sim(c,a))

0.0 0.7071067811865475 0.7071067811865475


5.2레반슈타인 거리

In [2]:
#어떻게하면 최소한의 수정단위를 거쳐서 처음단어에서 나중단어로 갈것인지 보여주는 거리
#추가, 삭제, 수정중 거리가 가장 짧은것을 return 추가, 삭제는 +1, 대각선위치면 그대로

def leven(text1, text2):
  len1 = len(text1) +1
  len2 = len(text2) +1

  sim_array = np.zeros((len1, len2)) #matrix테이블 만들기
  sim_array[:,0] = np.linspace(0, len1-1, len1) #모든행 첫열에 0~len1-1길이만큼 len1만큼 등분해서 숫자를 채워달라
  sim_array[0,:] = np.linspace(0, len2-1, len2) #0번째 행 모든 열에 마찬가지로 linspace만큼 채워달라
  for i in range(1, len1):
    for j in range(1, len2):
      add_char = sim_array[i-1,j] +1 # 추가는 위(row)의 값에 +1
      sub_char = sim_array[i,j-1] +1 # 삭제는 옆(column의 값에 +1 
      if text1[i-1] == text2[j-1]:
        mod_char = sim_array[i-1, j-1] #수정은 대각선에 있는것이 같으면 그대로, 다르면 +1 대각선 위치(i-1, j-1)
      else:
        mod_char = sim_array[i-1, j-1] + 1
      sim_array[i,j] = min([add_char, sub_char, mod_char]) # 추가, 삭제, 수정중 가장 짧은것을 return
  return sim_array[-1,-1]  # (-1,-1)의 위치 = array의 오른쪽 끝

print(leven('데이터마이닝', '데이타마닝'))



2.0


# 6.Word2Vec - CBoW, SkipGrim

6.1 CBow와 SkipGram을 위한 전처리 복습 및 Overview

In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
data = pd.read_csv('transcripts.csv')
print('Missing Values :' , data.isnull().sum())
data = data.dropna().reset_index(drop=True) 

#한줄씩 나눠진 data를 str로 받아서 합치는 작업 df에 iloc for문으로 할당 

merge_data = ''.join(str(data.iloc[i,0]) for i in range(100))
print('Total word count: ', len(merge_data))
print(merge_data[:40])

In [ ]:
tokenizer = RegexpTokenizer("[\w]+")
token_text = tokenizer.tokenize(merge_data)

#불용어제거

stop_words = set(stopwords.words('english'))
token_stop_text = []
for w in token_text:
  if w not in stop_words:
    token_stop_text.append(w)

print('After cleaning: ', len(token_stop_text))

In [ ]:
word2inx = {}
Bow = []
for word in token_stop_text:
  if word not in word2inx.keys():
    word2inx[word] = len(word2inx)
    Bow.insert(len(word2inx)-1,1)  #원하는 위치에 추가할때 insert 사용 
  else:
    inx = word2inx.get(word)
    Bow[inx] += 1

print('Unique Words Count :' , len(Bow))

6.2 nltk 내장 함수를 이용한 CBow 학습

In [ ]:
import numpy as np
token_stop_text = np.reshape(np.array(token_stop_text),[-1,1])
from gensim.models import Word2Vec

#vector size: 임베딩벡터의 사이즈
#sg : skip-gram은 1, CBow는 0
#min_count 최소 빈도 미만이면 제거
#window 참조할 주변 벡터 크기 

model = Word2Vec(vector_size = 100, window = 5 , min_count = 2 ,sg = 0)
model.build_vocab(token_stop_text)
model.train(token_stop_text, total_examples = model.corpus_count , epochs = 30, report_delay = 1)
vocabs = model.wv.key_to_index.keys() #단어의 임베딩 형태 확인
word_vec_list = [model.wv[i] for i in vocabs] #각 vocab의 imbedded vector확인
           

# 6.3 PCA를 통한 학습 모델 시각화

-16600개의 단어 각각 100개의 벡터가 임베딩 되어있음
-시각화를 위해 100차원을 ->2차원으로 축소

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 2)
pcafit = pca.fit_transform(word_vec_list)
x = pcafit[0:50,0]
y = pcafit[0:50, 1]

import matplotlib.pyplot as plt
plt.scatter(x, y, marker = 'o')
for i , v in enumerate(vocabs):
  if i <=49:
    plt.annotate(v , xy = (x[i], y[i]))

plt.show()

# 7. SGNS : SkipGram with Negative Sampling

7.1 SkipGram 전용 Dataset 구성

토큰화된 결과만 필요했던 CBOW 및 SkipGram과는 달리 SGNS는 두 단어의 인접 여부가 labeling에서 제공하는 전처리 도구를 활용
기본적인 토큰화 과정을 거친 후에 skipgram 함수를 이용하여 변환
단어 사이가 근접하면 1, 근접하지 않으면0
비슷한 애들끼리 임베딩 벡터의 차이를 작게하기 위함

In [ ]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import numpy as np

data = pd.read_csv('trancripts.csv')
print('Missing Values :', data.isnull().sum())
data = data.dropna().reset_index(drop=True)
merge_data = ''.join(str(data.iloc[i,0]) for i in range(30))
print('Total word count: ', len(merge_data))

tokenizer = RegexpTokenizer("[\w]+")
token_text = tokenizer.tokenize(merge_data)

stop_words = set(stopwords.words('english'))
token_stop_text = []
for w in token_text:
  if w not in stop_words:
    token_stop_text.append(w)

print('After cleaning :' , len(token_stop_text)

In [ ]:
from tensorflow.kears.preprocessing.text import tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token_stop_text)
word2inx = tokenizer.word_index
encoded = tokenizer.texts_to_sequences(token_stop_text)
encoded = np.array(encoded).T
from tensorflow.keras.preprocessing.sequence import skipgrams
skip_gram = [skipgrams(sample, vocabulary_size = len(word2inx)+1,
                       window_size= 10) for sample in encoded]
#dataset의 구성 : text1, text2, ... label
                     

In [ ]:
from urllib.request import FTPHandler
from traitlets.traitlets import Float
import torch
import torch.nn as nn
from torch import LongTensor as LT
from torch import FloatTensor as FT

Word2Vec의 원리는 각 단어마다 지정된 고유 벡터 값을 생성하는 것

In [ ]:
#nn.embedding(vocab, embed) : 정수 인코딩된 결과를 넣으면 word2vec 임베딩 결과를 나타내줌

class Word2Vec(nn.Module):
  def __init__(self, vocab_size, embed_size):
    super(Word2Vec, self).__init__()
    self.vocab_size = vocab_size
    self.embed_size = embed_size
    self.word1_vector = nn.Embedding(self.vocab_size, self.embed_size)
    self.word2_vector = nn.Embedding(self.vocab_size, self.embed_size)
    self.word1_vector.weight = nn.Parameter(torch.cat([torch.zeros(1, self.embed_size), FT(self.vocab_size-1,
                                                                                           self.embed_size).uniform(-0.1, 0.1)]))
    self.word2_vector.weight = nn.Parameter(torch.cat([torch.zeros(1, self.embed_size), FT(self.vocab_size-1,
                                                                                           self.embed_size).uniform(-0.1, 0.1)]))
    self.word1_vector.weight.requires_gard = True
    self.word2_vector.weight.requires_gard = True
  
# pytorch 사용하기 위해서 tensor에 래핑 LT
# GPU를 사용한다면 GPU도 래핑해줌 

  def forward_word1(self, data):
    vec = LT(data)
    vec = vec.cuda() if self.word1_vector.weight.is_cuda else vec
    return self.word1_vector(vec)

  def forward_word2(self, data):
    vec = LT(data)
    vec = vec.cuda() if self.word2_vector.weight.is_cuda else vec
    return self.word2_vector(vec)


torch.bmm

[B, n, m] x [B, m, p] = [B, n. p]

유사도가 비슷하다면 2개의 dot값이 작으면 좋겠음

word1 -> embed  1

word2 -> embed  2

1.2를 내적하고 싶음

[B , m]

[B , m]  이 둘에 차원을 추가하면 내적가능

ex> [1,m]x[mx1] -> [1,1] 스칼라 matrix : 벡터의 내적결과

word1 [B,m] -> [B.1.m]

word2 [B,m] -> [B,m,1]

label [B] -> [B,1]

추가된1은 unsqueeze 함수를 통해 찾기 가능

In [ ]:
from os import WEXITED
class SGNS(nn.Module):
  def __init__(self, embed, vocab_size):
    super(SGNS, self).__init__()
    slef.embed = embed
    self.vocab_size = vocab_size
    self.weights = None
  
  def forward(self, word1, word2, label):
    word1 = self.embed.forward_word1(word1).unsqueeze(1)
    word2 = self.embed.forward_word2(word2).unsqueeze(2)
    label = LT(label).unsqueeze(1)
    prediction = torch.bmm(word1, word2).squeeze(2).sigmoid().log()
    loss = -label * prediction
    return loss,mean()


#squeeze를 통해 추가된 차원을 없앨 수 있음 위는 [B.1.1]이었으나 squeeze를 통해 [B,1]


Q) torch.bmm의 계산 형태
  
*왜 MLP의 형태처럼 loss계산이 불가능한가?

*negative sampling을 어떻게 처리하고 있는가?

Q) 이 코드에서 loss로 정의된 term이 cross-entropy를 대신할 수 있는 이유는?

Q) .unsqueeze(), .squeeze() --> 차원 확장, 차원 축소의 역할

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDdataset
from tqdm import tqdm
vocab_size = len(word2idx)+1
word2vec = Word2Vec(vocab_size = vocab_size, embed_size = 100)
sgns = SGNS(embed = word2vec, vocab_size = vocab_size)
optim = Adam(sgns.parameters())
print('Train Ready')

In [ ]:
# data를 tensor로 래핑한 후 , dataloader로 불러옴

for _, element in enumerate(skip_gram):
  word1 = LT(np.array(list(zip(*element[0]))[0], dtype = 'int32'))
  word2 = LT(np.array(list(zip(*element[0]))[1], dtype = 'int32'))
  label = LT(np.array(element[1], dtype = 'int32'))
  dataset = TensorDataset(word1, word2, label)
  train_loader = DataLoader(dataset, batch_size = 256, shuffle = True) #미니배치 실행될때마다 shuffle

print('Data Loaded')  

In [ ]:
for epoch in range(5):
  with tqdm(train_loader, unit = 'batch') as tepoch:
    for word1, word2, label in tepoch:
      loss = sgns(word1, word2, label)
      optim.zero_grad()
      loss.backward()
      optim.step()
      tepoch.set_descroption(f"Epoch {epoch}")
      tepoch.set_postfix(loss = loss.item)
      
# label이 1인 애들의 내적이 줄어드는 방향으로 학습완료

In [ ]:
#학습 검증 위해 word2vec에 imbedding vector를 가져와서 확인
f = open('vectors.txt', 'w')
ww = 0
f.write('{} {}\n'.format(7930, 100))
vectors = word2vec.word1_vector.weight.detach().numpy()
for i, v in enumerate(word2idx.keys()):
  try:
    f.write('{} {}\n'.format(v, ' '.join(map(str, list(vectots[i+1, :])))))
    ww += 1
  except:
    continue

f.close()

import gensim
embed_word2vec = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary = False)


In [ ]:
embed_word2vec.most_similar(postive = ['obey'])

In [ ]:
embed_word2vec.most_similar(postive = ['love'])